# Out of the box visualisations
The purpose of this notebook is to provide some illustration about the data in the Panama papers. It's _out of the box_ in the sense that we only – mostly – only deal with data from the Panama Paper

## World-wide overview
Let see how many entities are on the panama paper linked with each country.

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium

In [2]:
entities = pd.read_csv('../data/panama/Entities.csv', header=0, low_memory=False)

In [3]:
# Remove entities with many country codes (leads to many difficulties)
filtered = entities[entities.country_codes.str.contains(';')==False]
data = filtered[['name', 'country_codes']].groupby('country_codes').count()

Now that we've the count per-country, let fill in the world countries with the data:

In [4]:
import json
data2 = []
with open('data/world-countries.json', 'r') as file:
    df = json.load(file)
    
for i,code in pd.DataFrame(df['features']).id.iteritems():
    try:
        v = data.loc[code]['name']
    except:
        pass
    
    data2.append([code,v])

NameError: name 'v' is not defined

In [ ]:
geo_path = 'data/world-countries.json'
map = folium.Map(location=[46.88, 8.23],tiles='Mapbox Bright',zoom_start=4.5)
map.choropleth(geo_path=geo_path, data=data2, fill_color='PuRd', key_on='feature.id',
               threshold_scale=[0, 10, 1000, 10000, 25000, 30000])
map.save('world.html')
map

Look at the swiss part of the data:

In [ ]:
from helpers.swiss_codes import filter_swiss, add_columns

swiss_entities = filter_swiss(entities)
data3 = add_columns(swiss_entities)

In [ ]:
cantons = pd.read_csv('data/cantons_codes.csv', header=None, names=['Name'])

In [ ]:
per_canton = data3[['name','canton']].groupby('canton').count()
for canton in cantons['Name']:
    if not canton in per_canton.index:
        per_canton.loc[canton] = 0
per_canton.sort_values(by='name', ascending=False).plot.bar(legend=False, figsize=(10,6))
plt.title('Number of entities per canton')
plt.ylabel('Number of entities (log scale)')
plt.yscale('log')
plt.show()

In [ ]:
geo_path = 'data/ch-cantons.topojson.json'
map = folium.Map(location=[46.88, 8.23],zoom_start=8)
map.choropleth(geo_path=geo_path, data_out='data2.json', data=per_canton.reset_index(),
               columns=['canton','name'],
               key_on='feature.id',
               fill_color='PuRd', fill_opacity=0.8, line_opacity=0.5,
               topojson='objects.cantons',
              threshold_scale=[0, 100, 500, 1000, 5000, 10000])
map.save('per_canton.html')
map